# 爬虫

In [1]:
import json
import re
import typing
from dataclasses import dataclass
from datetime import datetime, date
from concurrent.futures import ThreadPoolExecutor

from DrissionPage import ChromiumOptions, ChromiumPage
from DrissionPage.common import Settings
from DrissionPage.errors import ElementNotFoundError, WaitTimeoutError
from rich import print


In [1]:
import pandas as pd

# 读取原始 Excel 文件
input_file = "data_china.xlsx"  # 输入文件名
output_file = "group_control_prim.xlsx"  # 输出文件名

# 读取 Excel 数据
df = pd.read_excel(input_file,sheet_name=0,dtype={'股票代码': str})

# 筛选出 "真实性" 列值为 0 的行
filtered_df = df[df["真实性（虚假1真实0）"] == 0]

# 选择保留的列
columns_to_keep = ["股票代码", "公司简称", "新闻发布时间"]
filtered_df = filtered_df[columns_to_keep]

filtered_df["新闻发布时间"] = pd.to_datetime(filtered_df["新闻发布时间"]).dt.date
# 将筛选后的数据保存到新的 Excel 文件
filtered_df.to_excel(output_file, index=False)

print(f"已成功保存筛选后的数据到 {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: 'data_china.xlsx'

未爬取的代码

In [2]:
import os
import pandas as pd

# 文件夹路径
folder_path = "D:\mycodelife\workshop\\fake_finance\\ready_crawler"

# 获取文件夹中所有文件的文件名
files = os.listdir(folder_path)

# 假设文件名即为股票代码，去掉扩展名并存储为集合
crawled_codes = {os.path.splitext(file)[0] for file in files}

df = pd.read_excel("group_control_prim.xlsx",dtype={'股票代码': str})
# 股票代码列表（可以从一个 DataFrame 中读取，例如）
all_codes = df["股票代码"]  # 示例股票代码

# 找出未爬取的股票代码并输出

uncrawled_df = df[~df["股票代码"].isin(crawled_codes)]

# 保存未爬取股票代码的信息到新文件
output_file = "uncrawled_codes.xlsx"
uncrawled_df.to_excel(output_file, index=False)


In [3]:
df = pd.read_excel("uncrawled_codes.xlsx",dtype={'股票代码':str})
df

,股票代码,新闻发布时间,公司简称
0,600057,2015-01-09,象屿股份
1,600645,2015-10-22,中源协和
2,000673,2015-11-27,当代东方
3,600530,2016-04-25,交大昂立
4,600610,2017-02-13,中毅达
...,...,...,...
149,300565,2022-08-22,科信技术
150,688661,2022-09-01,和林微纳
151,002907,2022-09-02,华森制药
152,002752,2022-09-13,昇兴股份


In [5]:
import json
import re
import typing
from dataclasses import dataclass
from datetime import datetime, date
from concurrent.futures import ThreadPoolExecutor

from DrissionPage import ChromiumOptions, ChromiumPage
from DrissionPage.common import Settings
from DrissionPage.errors import ElementNotFoundError, WaitTimeoutError

DEBUG = False
DT_FMT = "%Y-%m-%d %H:%M:%S"

Settings.raise_when_wait_failed = True


@dataclass
class Article:
    title: str
    url: str
    published: datetime
    content: str


class PageNotFound(Exception):
    pass


def get_chrome_options() -> ChromiumOptions:
    co = ChromiumOptions()
    co.set_argument("--no-sandbox")
    if not DEBUG:
        co.headless()
        co.set_user_agent(
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
        )
    return co


class Fetcher:
    def __init__(self, code: str, date: date, page: ChromiumPage):
        self.code = code
        self.start_url = f"https://guba.eastmoney.com/list,{code}"
        self.date = date
        self.page = page
        self.total_page = -1

    def get_url(self, pn: int) -> str:
        return f"{self.start_url}_{pn}.html"

    def get_article_url(self, post_id: int) -> str:
        return f"https://guba.eastmoney.com/news,{self.code},{post_id}.html"

    def get_article_list(self, pn: int) -> typing.Generator[Article, None, None]:
        self.page.get(self.get_url(pn))
        #print(f"[green] Fetching page {self.get_url(pn)}...")
        article_list = re.search(
            r"var article_list\s*=\s*({.*?});", self.page.html
        ).group(1)

        json_data = json.loads(article_list)
        articles = json_data["re"]
        for article in articles:
            yield Article(
                title=article["post_title"],
                url=self.get_article_url(article["post_id"]),
                published=datetime.strptime(article["post_publish_time"], DT_FMT),
                content="",
            )

    def get_total_page(self):
        if self.total_page == -1:
            print(self.get_url(1))
            self.page.get(self.get_url(1), retry=3)
            if self.page.url == "https://guba.eastmoney.com/error?type=1":
                raise PageNotFound("Page not found")
            self.page.wait.ele_displayed("t:ul@class:paging")
            pagers = self.page.ele("t:ul@class:paging").eles("t:li")
            last_page = pagers[-2].text
            self.total_page = int(last_page)
            print(f"Total page: {self.total_page}")

    def get_article_detail(self, article: Article) -> Article:
        # print(f"[green] {article.published} {article.title}")
        new_tab = self.page.new_tab()
        try:
            new_tab.get(article.url)
            new_tab.wait.ele_displayed("@class:newstext")
            article.content = new_tab.ele("@class:newstext").text
            return article
        except:
            return article
        finally:
            new_tab.close()

    @staticmethod
    def get_date_range(articles: typing.Iterable[Article]) -> tuple[date, date]:
        articles = list(articles)

        return articles[0].published.date(), articles[-1].published.date()

    def get_articles_with_date(self) -> typing.Generator[Article, None, None]:
        self.get_total_page()

        # Binary search
        start, end = 1, self.total_page

        while start <= end:
            # print(f"[blue] Searching: {start} - {end}")
            mid = (start + end) // 2
            earliest_date_mid, latest_date_mid = self.get_date_range(
                self.get_article_list(mid)
            )

            if self.date > earliest_date_mid:
                end = mid - 1
            elif self.date < latest_date_mid:
                start = mid + 1
            else:
                end = mid - 1

        print(f"目标页面: {start}")

        # 获取目标页前后3天的文章
        start_page = start - 1 if start - 1 > 0 else 1
        end_page = end + 1 if end + 1 <= self.total_page else self.total_page

        # 未做优化，maxworker过大会爆内存
        with ThreadPoolExecutor(max_workers=2) as executor:
            for page in range(start_page, end_page + 1):
                # print(f"[blue] Fetching page {page}...")
                yield from executor.map(
                    self.get_article_detail, self.get_article_list(page)
                )


if __name__ == "__main__":
    import csv
    from openpyxl import load_workbook

    wb = load_workbook("uncrawled_codes.xlsx")
    ws = wb.active
    chrome_page = ChromiumPage(get_chrome_options())
    for row in ws.iter_rows(min_row=2, values_only=True):
        code, dt, _ = row
        dt = dt.date()

        output = f"{code}_{dt}.csv"
        print(f"Fetching {code} on {dt}...")
        fetcher = Fetcher(code, dt, chrome_page)

        with open(output, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["标题", "链接", "发布时间", "内容"])
            try:
                for article in fetcher.get_articles_with_date():
                    writer.writerow(
                        [article.title, article.url, article.published, article.content]
                    )
                   
            except PageNotFound:
                print(f"Page not found: {code}")
            except ElementNotFoundError:
                print(f"Element not found: {code}")
            except WaitTimeoutError:
                print(f"Wait timeout: {code}")

        # fetcher.page.quit()


Fetching 600057 on 2015-01-09...

https://guba.eastmoney.com/list,600057_1.html

Total page: 630

目标页面: 607

Fetching 600645 on 2015-10-22...

...

目标页面: 43

Fetching 300919 on 2022-11-04...

https://guba.eastmoney.com/list,300919_1.html

Total page: 428

目标页面: 200